In [3]:
import requests

In [12]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/"

def get_data(endpoint): 
    URL = f"{BASE_URL}{endpoint}.json"
    response = requests.get(URL, timeout=10)
    return response.json()

def get_top_story_ids(limit=10):
    return get_data("topstories")[:limit]

def get_story(story_id):
    return get_data(f"item/{story_id}")

In [ ]:
top_ids = get_top_story_ids(limit=10)
for idx, story_id in enumerate(top_ids, start=1):
    item = get_story(story_id)
    title = item.get("title", "<no title>")
    url = item.get("url", f"https://news.ycombinator.com/item?id={story_id}")
    score = item.get("score", 0)
    by = item.get("by", "unknown")
    print(f"{idx}. {title} ({score} points by {by})")
    print(f"{url}")

NameError: name 'get_item' is not defined